# (1): Image Manipulation

This project hinges on our ability to model and remove the bright emission from the circumstellar disk, to have a better chance at finding a faint CPD component hidden in its glare.  Such a model of that emission is itself of great interest to the community, since it will help facilitate comparisons between how this emission varies spatially among different populations of disks.

To develop some intuition about *what* you will be modeling, and *how* you will be doing that, it is important to become familiar with mm/radio images of disks and their general manipulation.  This part of the project should help you learn how to load .FITS data, extract their information, and plot, manipulate, and annotate the images.  This notebook will get you started, and leaves blank cells for you to develop code to achieve various goals.

**(A): Load a .FITS file:**

We will start the project with the disk around the young star GW Lup.  You can download the $\lambda = 1.3$ mm wavelength continuum image from the DSHARP data repository, and then load the image (a 2-D array) and the header (a dictionary of information about the image) with some standard Astropy tools.

In [8]:
import os, sys, time           # I basically always load these in every code I write.
import numpy as np             # Numpy is the basic numerics package in Python.
from astropy.io import fits    # .FITS file manipulation package  
import urllib.request          # package for fetching files off the internet

# disk target name
disk_name = 'GWLup'

# set this to your project directory (or however you like)
working_dir = '/Users/sandrews/Desktop/DSHARP_CPDs/'

# if you don't already have the datafile, download it
if not os.path.exists(working_dir+disk_name+'_continuum.fits'):
    dsharp_url = 'https://bulk.cv.nrao.edu/almadata/lp/DSHARP/images/'+disk_name+'_continuum.fits'
    print('Downloading .FITS image...')
    urllib.request.urlretrieve(dsharp_url, working_dir+disk_name+'_continuum.fits')
    print('...Download complete.')

# load the .FITS file into a 2-D array (image) and header dictionary (header)
data = fits.open(disk_name+'_continuum.fits')
image = np.squeeze(data[0].data)
header = data[0].header
print(image.shape)

(3000, 3000)


**(B) Explore the Data Contents:**

